In [5]:
import requests
from bs4 import BeautifulSoup

# 대상 URL
url = "https://onstar.jj.ac.kr/"

# 웹 페이지 요청
response = requests.get(url)
html = response.text

# BeautifulSoup 객체 생성
soup = BeautifulSoup(html, "html.parser")

# 특정 div 태그 가져오기 (예: class="target-class"인 div 찾기)
div_data = soup.find("div", class_="ChildFrame popup")  # 특정 클래스가 있는 div
# 또는 특정 ID
# div_data = soup.find("div", id="target-id")

# div 내부의 텍스트 추출
if div_data:
    print(div_data.text.strip())  # 필요하면 .text 대신 .get_text(strip=True) 사용
else:
    print("해당 div를 찾을 수 없습니다.")

해당 div를 찾을 수 없습니다.


In [4]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
import os
import json

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

In [7]:
# JSON 파일 로드
json_file = "programs.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)
print(data)

# 문서를 LangChain에서 사용할 형식으로 변환
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)
    documents.append(text)

# OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

{'프로그램_정보': [{'주제': '기업분석과 취업 준비 전략', '내용': '기업분석과 재무분석의 기초, SWOT 분석과 전략적 사고 훈련, 직무분석과 자기역량 개발', '기간': '2. 11.(화) 14:00~17:00(3시간)', '수강 장소': '실시간 강의(zoom)', '특이사항': ['강의 진행 전 링크 공유 예정', '카메라 얼굴 비춰야 출석 인정, 이동하면서 교육 수강 불가'], '문의처': '063-220-4654'}, {'교육기간': '2025.02.11 ~ 2025.02.11', '신청기간': '2024.12.30 ~ 2025.02.07', '신청대상': '재학생(휴/복학생 포함), 2년내 졸업자, 도내청년층', '교육장소': '가상공간 온라인(실시간)강의실(100)', '참여혜택': '비교과 포인트 인정, 점프업 자기주도형 포인트 3점', '신청구분': '개인', '문의처': '063-220-4654'}]}

C:\Users\user\AppData\Local\Temp\ipykernel_27464\934141581.py:14: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


In [14]:
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI

# GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# 사용자 질문을 처리하는 함수
def search_program_info(query):
    response = qa_chain.run(query)
    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# 테스트 실행
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))


🔍 '2월에 참여할 수 있는 비교과 프로그램 추천해줘' 관련 정보:

 2024����도 �������� ���� ����� ����� 3��(면�� �����니���이션 전��) ���는 JJ ������������� Day(2��)를 추��합니다.


In [19]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

# ✅ JSON 파일 로드 (UTF-8로 강제 변환하여 한글 깨짐 방지)
json_file = "programs.json"
with open("programs.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("programs.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # ensure_ascii=False -> 한글 깨짐 방지
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 사용자 질문을 처리하는 함수 (결과 한글 깨짐 방지)
def search_program_info(query):
    response = qa_chain.run(query)
    
    # 🔥 UTF-8로 강제 디코딩하여 한글 깨짐 방지
    try:
        response = response.encode('utf-8').decode('utf-8')
    except UnicodeDecodeError:
        response = "❌ 결과를 디코딩하는 과정에서 문제가 발생했습니다."

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))

🔍 '2월에 참여할 수 있는 비교과 프로그램 추천해줘' 관련 정보:

 2024����도 �������� ���� ����� ����� 3��(면�� �����니���이션 전��) 프로그��이 2�� 13일에 ����되는 것으로 나와 있습니다. 이 프로그��은 비��과 포인트 인정과 ������� 자기주도�� 포인트 3��을 제공하��, ����생, 2�� 내 �����자, ��내 �������을 대상으로 ��니다. ��여를 ��하시면 063-220-4654로 문의하시면 ��니다.


In [18]:
def search_program_info(query):
    response = qa_chain.run(query)
    
    # 한글 깨짐 방지: 불필요한 인코딩 변환 제거 & 디코딩 오류 방지
    try:
        response = response.encode("utf-8", errors="ignore").decode("utf-8")
    except UnicodeDecodeError:
        response = "❌ 결과를 디코딩하는 과정에서 문제가 발생했습니다."

    return f"🔍 '{query}' 관련 정보:\n\n{response}"


In [20]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

# ✅ JSON 파일 로드 (UTF-8 강제 변환)
json_file = "programs_fixed.json"  # 인코딩 변환한 JSON 사용
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # ensure_ascii=False -> 한글 깨짐 방지
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 사용자 질문을 처리하는 함수 (결과 한글 깨짐 방지)
def search_program_info(query):
    response = qa_chain.run(query)
    
    # 한글 깨짐 방지: 불필요한 인코딩 변환 제거 & 디코딩 오류 방지
    try:
        response = response.encode("utf-8", errors="ignore").decode("utf-8")
    except UnicodeDecodeError:
        response = "❌ 결과를 디코딩하는 과정에서 문제가 발생했습니다."

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))

🔍 '2월에 참여할 수 있는 비교과 프로그램 추천해줘' 관련 정보:

 2024����도 �������� ���� ����� ����� 3��(면�� �����니���이션 전��) 프로그��이 2�� 13일에 ����되는 것으로 보입니다. 해당 프로그��은 비��과 포인트 인정 �� ������� 자기주도�� 포인트 3��을 제공하��, ����생, 2�� 내 �����자, ��내 �������을 대상으로 ��니다. 문의��는 063-220-4654입니다.


In [21]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")

# ✅ UTF-8로 변환한 JSON 파일 사용
json_file = "programs_fixed.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # 한글 깨짐 방지
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 사용자 질문을 처리하는 함수 (결과 한글 깨짐 방지)
def search_program_info(query):
    response = qa_chain.run(query)

    # 🔥 한글 깨짐 방지: JSON 변환 없이 그대로 출력
    try:
        response = str(response)  # UTF-8 강제 변환
    except UnicodeDecodeError:
        response = "❌ 결과를 디코딩하는 과정에서 문제가 발생했습니다."

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))

🔍 '2월에 참여할 수 있는 비교과 프로그램 추천해줘' 관련 정보:

 2024����도 �������� ���� ����� ����� 3��(면�� �����니���이션 전��) 프로그��이 2�� 13일에 ����되는 것으로 보입니다. 해당 프로그��은 비��과 포인트 인정 �� ������� 자기주도�� 포인트 3��을 제공하��, ����생, 2�� 내 �����자, ��내 �������을 대상으로 ��니다. 문의��는 063-220-4654입니다.


In [22]:
import sys
print(sys.getdefaultencoding())  # 기본 인코딩 확인
print(sys.stdout.encoding)  # 출력 인코딩 확인

utf-8
UTF-8


In [23]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# ✅ UTF-8로 변환한 JSON 파일 사용
json_file = "programs_fixed.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # ensure_ascii=False 적용
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 한글 깨짐 방지: UTF-8 강제 적용
import sys
import io
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='utf-8')

# ✅ 사용자 질문을 처리하는 함수
def search_program_info(query):
    response = qa_chain.run(query)

    # 🔥 한글 깨짐 방지
    try:
        response = response.encode('utf-8').decode('utf-8')  
    except UnicodeDecodeError:
        response = "❌ 결과 디코딩 오류 발생"

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))

UnsupportedOperation: detach

In [24]:
import os
import json
import sys
import io
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# ✅ Python 출력 인코딩 UTF-8로 설정 (detach 문제 해결)
if hasattr(sys.stdout, "reconfigure"):
    sys.stdout.reconfigure(encoding="utf-8")
else:
    sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding="utf-8")

print("✅ 출력 인코딩 UTF-8로 변경 완료!")

# ✅ UTF-8로 변환한 JSON 파일 사용
json_file = "programs_fixed.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # 한글 깨짐 방지
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 사용자 질문을 처리하는 함수
def search_program_info(query):
    response = qa_chain.run(query)

    # 🔥 한글 깨짐 방지
    try:
        response = response.encode("utf-8").decode("utf-8")  
    except UnicodeDecodeError:
        response = "❌ 결과 디코딩 오류 발생"

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))

AttributeError: 'OutStream' object has no attribute 'buffer'

In [25]:
import os
import json
import sys
import io
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.llms import OpenAI
from langchain.chains import RetrievalQA

# ✅ .env 파일 로드
load_dotenv()

# ✅ 환경 변수 설정
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# ✅ Jupyter Notebook인지 확인 후 UTF-8 인코딩 설정
def is_jupyter_notebook():
    try:
        from IPython import get_ipython
        return get_ipython() is not None
    except ImportError:
        return False

if is_jupyter_notebook():
    sys.stdout = sys.__stdout__  # Jupyter에서는 기본 출력 유지
    print("✅ Jupyter Notebook 환경에서 UTF-8 설정 적용 완료!")
else:
    if hasattr(sys.stdout, "reconfigure"):
        sys.stdout.reconfigure(encoding="utf-8")
    else:
        sys.stdout = io.TextIOWrapper(sys.stdout.buffer, encoding="utf-8")
    print("✅ 일반 Python 환경에서 UTF-8 설정 적용 완료!")

# ✅ UTF-8로 변환한 JSON 파일 사용
json_file = "programs_fixed.json"
with open(json_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# ✅ 문서를 LangChain에서 사용할 형식으로 변환 (UTF-8 인코딩 적용)
documents = []
for program in data["프로그램_정보"]:
    text = json.dumps(program, ensure_ascii=False)  # 한글 깨짐 방지
    documents.append(text)

# ✅ OpenAI 임베딩 모델 사용
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

# ✅ Chroma 벡터스토어에 저장
vectorstore = Chroma.from_texts(documents, embeddings)

# ✅ 검색용 Retriever 생성
retriever = vectorstore.as_retriever()

# ✅ GPT-4 모델 연결
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

# ✅ RAG 검색 기반 QA 시스템 구축
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# ✅ 사용자 질문을 처리하는 함수
def search_program_info(query):
    response = qa_chain.run(query)

    # 🔥 한글 깨짐 방지
    try:
        response = response.encode("utf-8").decode("utf-8")  
    except UnicodeDecodeError:
        response = "❌ 결과 디코딩 오류 발생"

    return f"🔍 '{query}' 관련 정보:\n\n{response}"

# ✅ 테스트 실행 (한글 깨짐 확인)
print(search_program_info("2월에 참여할 수 있는 비교과 프로그램 추천해줘"))